In [ ]:
# Only uncomment and run once in your terminal.  If you're running as a notebook, you may need to reload this window for this to work.

# !conda env create -f ./envs/audio-to-midi-environment.yml
# !conda activate audio-to-midi

In [ ]:
!pip install basic-pitch
!pip install music21
!pip install librosa
!pip install midiutil
!pip install pretty_midi

In [ ]:
import glob
import librosa
from basic_pitch.inference import predict_and_save, Model
from basic_pitch import ICASSP_2022_MODEL_PATH

basic_pitch_model = Model(ICASSP_2022_MODEL_PATH)

audio_directory = '../audio/'
audio_files = glob.glob(audio_directory + '*.mp3')

for audio_file in audio_files:
    
    # Estimate BPM using librosa
    print(audio_file)
    y, sr = librosa.load(audio_file)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    
    tempo = int(round(tempo[0]))
    print(f"Tempo: {tempo}")

    predict_and_save(
        audio_path_list=[audio_file],
        output_directory='../midi-out/',
        save_model_outputs=True,
        save_midi=True,
        sonify_midi=True,
        save_notes=True,
        minimum_note_length=127.70,
        model_or_model_path=basic_pitch_model,
        onset_threshold=0.4,
        frame_threshold=0.2,
        sonification_samplerate=44100,
        midi_tempo=tempo  # Use estimated tempo here
    )

In [5]:
import music21
import matplotlib.pyplot as plt
%matplotlib inline

midi_file = music21.converter.parse('../midi/Cello_basic_pitch.mid')

In [ ]:
midi_file.show('midi')

In [ ]:
midi_file.plot()

In [19]:
import os
import glob
import shutil
import librosa
import numpy as np
import pretty_midi
from basic_pitch.inference import predict_and_save, Model
from basic_pitch import ICASSP_2022_MODEL_PATH

def get_instrument_program(stem_name):
    return {
        'bass': 33,
        'drums': 0,
        'guitar': 25,
        'piano': 0,
        'vocals': 53,
        'other': 48,
    }.get(stem_name.lower(), 0)

def convert_stems_to_midi(stems_directory, output_directory):
    # Get track name from directory
    track_name = os.path.basename(stems_directory)
    print(f"\nProcessing track: {track_name}")
    
    # Initialize model
    model = Model(ICASSP_2022_MODEL_PATH)
    
    # Setup directories
    os.makedirs(output_directory, exist_ok=True)
    temp_dir = os.path.join(output_directory, 'temp')
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)  # Clean start
    os.makedirs(temp_dir)

    # Get all MP3 files
    mp3_files = glob.glob(os.path.join(stems_directory, '*.mp3'))
    if not mp3_files:
        print(f"No MP3 files found in {stems_directory}")
        return False
        
    print(f"Found {len(mp3_files)} MP3 files")

    # Process each stem
    tempos = []
    for mp3_file in mp3_files:
        stem_name = os.path.splitext(os.path.basename(mp3_file))[0]
        print(f"Processing stem: {stem_name}")
        
        # Detect tempo
        y, sr = librosa.load(mp3_file)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        tempo = int(tempo)
        tempos.append(tempo)
        
        # Convert to MIDI
        predict_and_save(
            audio_path_list=[mp3_file],
            output_directory=temp_dir,
            save_midi=True,
            sonify_midi=False,
            save_model_outputs=True,
            save_notes=True,
            minimum_note_length=127.70,
            model_or_model_path=model,
            onset_threshold=0.5,
            frame_threshold=0.3,
            sonification_samplerate=44100,
            midi_tempo=tempo
        )

    # Create combined MIDI
    final_tempo = int(np.median(tempos))
    combined_midi = pretty_midi.PrettyMIDI(initial_tempo=final_tempo)
    
    # Add each stem
    for mp3_file in mp3_files:
        stem_name = os.path.splitext(os.path.basename(mp3_file))[0]
        midi_file = os.path.join(temp_dir, f"{stem_name}_basic_pitch.mid")
        
        print(f"Adding stem: {stem_name}")
        
        try:
            stem_midi = pretty_midi.PrettyMIDI(midi_file)
            program = get_instrument_program(stem_name)
            
            if 'drums' in stem_name.lower():
                instrument = pretty_midi.Instrument(program=0, is_drum=True, name=stem_name)
                instrument.channel = 9
            else:
                instrument = pretty_midi.Instrument(program=program, name=stem_name)
                
            note_count = 0
            for note in stem_midi.instruments[0].notes:
                instrument.notes.append(pretty_midi.Note(
                    velocity=note.velocity,
                    pitch=note.pitch,
                    start=note.start,
                    end=note.end
                ))
                note_count += 1
            
            print(f"Added {note_count} notes")
            combined_midi.instruments.append(instrument)
            
        except Exception as e:
            print(f"Error processing {stem_name}: {str(e)}")
            continue

    # Save combined file with track name
    output_file = os.path.join(output_directory, f"{track_name}.mid")
    print(f"Saving as: {output_file}")
    
    try:
        combined_midi.write(output_file)
        print(f"Successfully created: {output_file}")
        success = True
    except Exception as e:
        print(f"Error saving combined MIDI: {str(e)}")
        success = False
    
    # Cleanup temp directory
    try:
        shutil.rmtree(temp_dir)
        print("Cleaned up temporary files")
    except Exception as e:
        print(f"Error cleaning up temp directory: {str(e)}")
    
    return success

def process_all_tracks(base_directory, output_directory):
    """Process all track folders in the base directory"""
    
    # Get all subdirectories
    track_dirs = [d for d in glob.glob(os.path.join(base_directory, '*')) 
                 if os.path.isdir(d)]
    
    print(f"Found {len(track_dirs)} tracks to process")
    
    # Process each track
    successful = 0
    failed = 0
    
    for track_dir in track_dirs:
        try:
            if convert_stems_to_midi(track_dir, output_directory):
                successful += 1
            else:
                failed += 1
        except Exception as e:
            print(f"Error processing track {track_dir}: {str(e)}")
            failed += 1
    
    print(f"\nProcessing complete!")
    print(f"Successfully processed: {successful} tracks")
    print(f"Failed to process: {failed} tracks")

if __name__ == "__main__":
    process_all_tracks(
        base_directory='../audio/stems/htdemucs_6s',
        output_directory='../midi-out'
    )

Found 2 tracks to process

Processing track: Cello
Found 6 MP3 files
Processing stem: guitar


/var/folders/wc/wntr3k4n1_xdk3mtbmwhbzjc0000gn/T/ipykernel_6764/1308284477.py:52: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tempo = int(tempo)



Predicting MIDI for ../audio/stems/htdemucs_6s/Cello/guitar.mp3...
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True
shape: (1, 43844, 1)
dtype: float32
isfinite: True